<a href="https://colab.research.google.com/github/flaviare1s/magazine-capivara/blob/main/Magazine_Capivara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Magazine Capivara

## Imports das Bibliotecas

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import dos dados

In [2]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTXDffy5v8dZQdPVO0oZYU-m4DAKYQa5wIO7QvTudUUaDt7gwEInmB5mTmbTxrRjCSqB6zwSr6XoWhA/pub?output=csv')
df.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
0,2023-01-01,19352,6031,13321,76,8,Roupas,1,Oeste
1,2023-01-01,47585,29758,17827,7,13,Móveis,1,Leste
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
4,2023-01-03,8644,4939,3705,33,1,Móveis,3,Norte


## Conexão com o Google Cloud

In [3]:
from google.colab import auth
auth.authenticate_user()
project_id = 'flavia-ad13'
!gcloud config set project {project_id}

from google.cloud import storage
client = storage.Client()
bucket_name = 'capivara-magazine2025'
bucket = client.bucket(bucket_name)

df.to_csv('capivara-bruta.csv', index=False)
destination_blob_name = 'dados-brutos/capivara-bruta.csv'
source_file_name = 'capivara-bruta.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

Updated property [core/project].


## 1. Divisão em Categorias

### Descobrindo quais categorias existem

In [4]:
categorias_unicas = df["Categoria do Produto"].unique()
print(categorias_unicas)

['Roupas' 'Móveis' 'Eletrônicos' 'Brinquedos' 'Alimentos']


### Separando em DataFrames por categoria

In [5]:
dfs_por_categoria = {categoria: df[df["Categoria do Produto"] == categoria] for categoria in df["Categoria do Produto"].unique()}

In [6]:
df_roupas = dfs_por_categoria["Roupas"]
df_moveis = dfs_por_categoria["Móveis"]
df_eletronicos = dfs_por_categoria["Eletrônicos"]
df_brinquedos = dfs_por_categoria["Brinquedos"]
df_alimentos = dfs_por_categoria["Alimentos"]

### Salvando em CSV e enviando para o GCP

In [9]:
df_roupas.to_csv('capivara-bruta-roupas.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-roupas.csv'
source_file_name = 'capivara-bruta-roupas.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [10]:
df_moveis.to_csv('capivara-bruta-moveis.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-moveis.csv'
source_file_name = 'capivara-bruta-moveis.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [11]:
df_eletronicos.to_csv('capivara-bruta-eletronicos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-eletronicos.csv'
source_file_name = 'capivara-bruta-eletronicos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [12]:
df_brinquedos.to_csv('capivara-bruta-brinquedos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-brinquedos.csv'
source_file_name = 'capivara-bruta-brinquedos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [13]:
df_alimentos.to_csv('capivara-bruta-alimentos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-alimentos.csv'
source_file_name = 'capivara-bruta-alimentos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## 2. Identificação dos produtos mais lucrativos

### Identificando as categorias que geraram mais lucro

In [67]:
lucro_total_por_categoria = df.groupby("Categoria do Produto")["Lucro"].sum().sort_values(ascending=False)
display(lucro_total_por_categoria)

,Lucro
Categoria do Produto,
Alimentos,1695177
Brinquedos,1631946
Eletrônicos,1622365
Roupas,1342888
Móveis,1288416


### Calculando a média geral de lucro

In [20]:
media_geral_lucro = df["Lucro"].mean().round(2)
print(media_geral_lucro)

12634.65


### Filtrando os produtos com lucro acima da média

In [22]:
df_lucro_acima_media = df[df["Lucro"] > media_geral_lucro]
df_lucro_acima_media.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
0,2023-01-01,19352,6031,13321,76,8,Roupas,1,Oeste
1,2023-01-01,47585,29758,17827,7,13,Móveis,1,Leste
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
9,2023-01-06,35695,15800,19895,75,2,Alimentos,5,Leste


### Enviando o DataFrame de produtos com lucro acima da média para o bucket no Google Cloud

In [23]:
df_lucro_acima_media.to_csv('produtos-com-lucro-acima-da-media.csv', index=False)
destination_blob_name = 'lucros/produtos-com-lucro-acima-da-media.csv'
source_file_name = 'produtos-com-lucro-acima-da-media.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## 3. Avaliação da Satisfação do Cliente

### Criando um DataFrame apenas com 'Satisfação do Cliente' nível 5

In [24]:
df_satisfacao_5 = df[df["Satisfação do Cliente"] == 5]
df_satisfacao_5.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
7,2023-01-05,43442,32811,10631,32,20,Alimentos,5,Oeste
9,2023-01-06,35695,15800,19895,75,2,Alimentos,5,Leste
10,2023-01-07,2008,1245,763,58,41,Eletrônicos,5,Norte


### Comparando os faturamentos

In [25]:
faturamento_medio_geral = df["Receita"].mean()
print(f"Faturamento Médio Geral: {faturamento_medio_geral:.2f}")

Faturamento Médio Geral: 25647.40


In [26]:
faturamento_medio_satisfacao_5 = df_satisfacao_5["Receita"].mean()
print(f"Faturamento Médio (Satisfação 5): {faturamento_medio_satisfacao_5:.2f}")

Faturamento Médio (Satisfação 5): 26798.24


In [27]:
if faturamento_medio_satisfacao_5 > faturamento_medio_geral:
    print("O faturamento médio das vendas com Satisfação 5 é maior que o faturamento médio geral.")
else:
    print("O faturamento médio das vendas com Satisfação 5 é menor que o faturamento médio geral.")

O faturamento médio das vendas com Satisfação 5 é maior que o faturamento médio geral.


### Enviando para o Google Cloud

In [28]:
df_satisfacao_5.to_csv('satisfacao-maxima.csv', index=False)
destination_blob_name = 'satisfacao/satisfacao-maxima.csv'
source_file_name = 'satisfacao-maxima.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## 5. Novos Clientes e Expansão do Negócio

## Região com maior número de novos clientes

In [66]:
novos_clientes_por_regiao = df.groupby("Região")["Novos Clientes"].sum().sort_values(ascending=False)
display(novos_clientes_por_regiao)

,Novos Clientes
Região,
Norte,3881
Leste,3692
Sul,3454
Oeste,3317


### Criando um DataFrame apenas com os dados da região Norte

In [30]:
df_regiao_norte = df[df["Região"] == "Norte"]
df_regiao_norte.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
4,2023-01-03,8644,4939,3705,33,1,Móveis,3,Norte
6,2023-01-04,3846,382,3464,79,6,Roupas,1,Norte
10,2023-01-07,2008,1245,763,58,41,Eletrônicos,5,Norte
11,2023-01-07,48525,46889,1636,87,38,Móveis,1,Norte


### Enviando para o GCP

In [31]:
df_regiao_norte.to_csv('regiao-norte.csv', index=False)
destination_blob_name = 'novos_clientes/regiao-norte.csv'
source_file_name = 'regiao-norte.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## Insights

1. Categorias Mais Lucrativas
Pergunta: Quais categorias de produtos geram mais lucro?
R - Alimentos, brinquedos e eletrônicos.

2. Satisfação do Cliente x Lucro
Pergunta: Os clientes mais satisfeitos geram mais lucro?
R - Sim.

3. Novos Clientes e Crescimento
Pergunta: Quais regiões estão trazendo mais novos clientes?
R - Norte em primeiro lugar. Leste em segundo.

4. Regiões Mais Lucrativas
Pergunta: Qual região tem maior faturamento e lucro? R - Região Norte.

In [32]:
df.groupby("Região")["Receita"].sum().sort_values(ascending=False)

,Receita
Região,
Norte,4222193
Sul,3904066
Oeste,3640511
Leste,3621671


In [33]:
df.groupby("Região")["Lucro"].sum().sort_values(ascending=False)

,Lucro
Região,
Norte,2037041
Sul,1968494
Oeste,1908531
Leste,1666726


5. Quantidade de Vendas x Lucro
Pergunta: Mais vendas sempre significam mais lucro? R - Apesar de a região Norte ter o maior número de vendas e a maior lucratividade també, o cálculo de correlação entre essas duas variáveis deu 0.02, indicando que não há uma correlação clara.

In [34]:
df.groupby("Região")["Quantidade de Vendas"].sum().sort_values(ascending=False)

,Quantidade de Vendas
Região,
Norte,8324
Leste,7342
Oeste,7311
Sul,6357


In [35]:
correlacao = df["Quantidade de Vendas"].corr(df["Lucro"])
print(f"Correlação entre Quantidade de Vendas e Lucro: {correlacao:.2f}")

Correlação entre Quantidade de Vendas e Lucro: 0.02


6. Impacto de Promoções e Datas Específicas
Pergunta: Alguma data teve um aumento significativo de vendas? R - Sim, algumas datas tiveram vendas elevadas, sendo o dia 12/06/2023 o que mais vendeu.

In [65]:
df_vendas_por_data = df.groupby("Data")["Quantidade de Vendas"].sum().sort_values(ascending=False)
display(df_vendas_por_data)

,Quantidade de Vendas
Data,
2023-06-12,181
2023-09-19,180
2023-03-10,177
2023-09-25,174
2023-02-24,172
...,...
2023-07-20,6
2023-10-18,5
2023-01-30,4


7. Custos Elevados
Pergunta: Alguma categoria está com custo muito alto? R - A categoria 'Brinquedos' tem custo mais elevados do que as outras.

In [64]:
custo_por_categoria = df.groupby("Categoria do Produto")["Custo"].sum().sort_values(ascending=False)
display(custo_por_categoria)

,Custo
Categoria do Produto,
Brinquedos,1930154
Alimentos,1657990
Móveis,1529759
Eletrônicos,1459933
Roupas,1229813


8. Impacto do Volume de Vendas no Lucro
Pergunta: Produtos com mais unidades vendidas são os mais lucrativos? R - O Cálculo realizado indicou uma correlação positiva moderada.

In [45]:
df_produtos = df.groupby("Categoria do Produto")[["Quantidade de Vendas", "Lucro"]].sum()

correlacao = df_produtos["Quantidade de Vendas"].corr(df_produtos["Lucro"])

print("Correlação entre vendas e lucro:", correlacao)

Correlação entre vendas e lucro: 0.6684913658697638


9. Efeito de Altas e Baixas no Lucro Diário
Pergunta: Há dias com lucro extremamente alto ou baixo? R - Sim.

In [63]:
df_lucro_por_data = df.groupby("Data")["Lucro"].sum().sort_values(ascending=False)
display(df_lucro_por_data)

,Lucro
Data,
2023-11-22,78105
2023-11-19,71276
2023-11-08,70010
2023-11-11,69814
2023-01-21,66350
...,...
2023-12-05,104
2023-07-31,35
2023-05-24,31


10.  Conversão de Novos Clientes
Pergunta: Novos clientes geram um faturamento relevante? R  Sim, pois o faturamento médio por novo cliente é de 1072,81.

In [56]:
faturamento_total = df["Receita"].sum()

faturamento_por_novos_clientes = df.groupby("Novos Clientes")["Receita"].sum()

total_novos_clientes = df["Novos Clientes"].sum()
faturamento_medio_por_cliente = faturamento_total / total_novos_clientes

print(f"Faturamento total: {faturamento_total}")
print(f"Faturamento médio por novo cliente: {faturamento_medio_por_cliente:.2f}")

Faturamento total: 15388441
Faturamento médio por novo cliente: 1072.81


11. Distribuição de Lucro por Categoria e Região
Pergunta: Alguma região tem um desempenho excepcional para certos produtos? R - Sim, alimentos nas regiões Sul e Oeste são os maiores lucros.

In [62]:
lucro_por_categoria_regiao = df.groupby(["Categoria do Produto", "Região"])["Lucro"].sum().reset_index().sort_values(by="Lucro", ascending=False)
display(lucro_por_categoria_regiao)

,Categoria do Produto,Região,Lucro
3,Alimentos,Sul,525686
2,Alimentos,Oeste,494265
5,Brinquedos,Norte,492566
9,Eletrônicos,Norte,487549
7,Brinquedos,Sul,467328
8,Eletrônicos,Leste,440609
10,Eletrônicos,Oeste,406591
18,Roupas,Oeste,391040
15,Móveis,Sul,384294
13,Móveis,Norte,383303


12. Relação entre Receita e Satisfação do Cliente
Pergunta: Os produtos mais vendidos são bem avaliados? R - Embora os produtos com avaliação 5 tenham sido os mais vendidos, em segundo lugar ficaram os produtos com avaliação 1, mostrando que os produtos com baixa avaliação também têm grande relevância no total de vendas. Isso indica que a popularidade de um produto não necessariamente reflete sua qualidade ou a satisfação dos clientes, pois a avaliação negativa também tem um peso considerável nas vendas.

In [75]:
relacao_receita_vendas_satisfacao = df.groupby("Satisfação do Cliente").agg(
    receita_total=("Receita", "sum"),
    vendas_total=("Quantidade de Vendas", "sum")
).sort_values(by="receita_total", ascending=False)

display(relacao_receita_vendas_satisfacao)

,receita_total,vendas_total
Satisfação do Cliente,,
5,3403377,6161
1,3262113,6070
4,2955379,6004
2,2899622,5125
3,2867950,5974


13. Regiões com Alto Custo Operacional
Pergunta: Alguma região tem um custo muito elevado? R - A região Norte

In [78]:
custo_por_regiao = df.groupby(["Região"])["Custo"].sum().reset_index().sort_values(by="Custo", ascending=False)
display(custo_por_regiao)

,Região,Custo
1,Norte,2185152
0,Leste,1954945
3,Sul,1935572
2,Oeste,1731980


14. Proporção de Custo x Receita por Categoria
Pergunta: Quais categorias têm os melhores e piores retornos sobre o investimento? R - Eletrônicos tem o melhor retorno e móveis tem o pior.

In [81]:
df_categoria = df.groupby("Categoria do Produto").agg(
    receita_total=("Receita", "sum"),
    custo_total=("Custo", "sum")
).reset_index()

df_categoria["retorno_investimento"] = df_categoria["receita_total"] / df_categoria["custo_total"]

df_categoria = df_categoria.sort_values(by="retorno_investimento", ascending=False)

display(df_categoria)

,Categoria do Produto,receita_total,custo_total,retorno_investimento
2,Eletrônicos,3082298,1459933,2.111260
4,Roupas,2572701,1229813,2.091945
0,Alimentos,3353167,1657990,2.022429
1,Brinquedos,3562100,1930154,1.845500
3,Móveis,2818175,1529759,1.842235


15. Concentração de Vendas por Região
Pergunta: Alguma região domina as vendas de certos produtos? R - Sim, a região Norte, por exemplo, detém quase 38% da venda de eletrônicos.


In [82]:
df_vendas_por_regiao_categoria = df.groupby(["Região", "Categoria do Produto"])["Quantidade de Vendas"].sum().reset_index()

df_vendas_por_categoria = df.groupby("Categoria do Produto")["Quantidade de Vendas"].sum().reset_index()
df_vendas_por_categoria = df_vendas_por_categoria.rename(columns={"Quantidade de Vendas": "Total por Categoria"})

df_regiao_categoria = pd.merge(df_vendas_por_regiao_categoria, df_vendas_por_categoria, on="Categoria do Produto")

df_regiao_categoria["Participação (%)"] = (df_regiao_categoria["Quantidade de Vendas"] / df_regiao_categoria["Total por Categoria"]) * 100

df_regiao_categoria = df_regiao_categoria.sort_values(by="Participação (%)", ascending=False)

display(df_regiao_categoria)

,Região,Categoria do Produto,Quantidade de Vendas,Total por Categoria,Participação (%)
7,Norte,Eletrônicos,2358,6253,37.709899
6,Norte,Brinquedos,2057,6182,33.274021
14,Oeste,Roupas,1435,4690,30.597015
15,Sul,Alimentos,1882,6261,30.059096
10,Oeste,Alimentos,1876,6261,29.963265
3,Leste,Móveis,1690,5948,28.412912
4,Leste,Roupas,1313,4690,27.995736
8,Norte,Móveis,1639,5948,27.555481
2,Leste,Eletrônicos,1655,6253,26.467296
18,Sul,Móveis,1546,5948,25.991930


16. Análise de Tendências por Categoria
Pergunta: Alguma categoria está crescendo ou diminuindo em vendas ao longo do tempo?

In [94]:
df['Data'] = pd.to_datetime(df['Data'])

df_vendas_por_categoria_data = df.groupby([df['Data'].dt.to_period('M'), 'Categoria do Produto'])['Quantidade de Vendas'].sum().reset_index()

df_vendas_por_categoria_data['Variação (%)'] = df_vendas_por_categoria_data.groupby('Categoria do Produto')['Quantidade de Vendas'].pct_change() * 100

df_vendas_por_categoria_data_sorted = df_vendas_por_categoria_data.sort_values(by='Variação (%)', ascending=False)

display(df_vendas_por_categoria_data_sorted)

,Data,Categoria do Produto,Quantidade de Vendas,Variação (%)
49,2023-10,Roupas,644,287.951807
23,2023-05,Móveis,562,255.696203
38,2023-08,Móveis,806,131.609195
11,2023-03,Brinquedos,504,122.026432
58,2023-12,Móveis,657,115.409836
55,2023-12,Alimentos,821,105.250000
40,2023-09,Alimentos,650,94.029851
39,2023-08,Roupas,178,61.818182
15,2023-04,Alimentos,538,52.407932
27,2023-06,Eletrônicos,552,50.000000
